In [1]:
import numpy as np
import pandas as pd


In [2]:
x1 = np.random.randint(100,size=(20))
x2 = np.random.randint(100,size=(20))
x3 = np.random.randint(100,size=(20))


y1 = 2*x1 + 3*x2 + 4*x3

y2 = 7*x1 - 2*x2 + x3

In [3]:
XX1 =  {'col1': x1, 'col2': x2, 'col3': x3}
XX1 = pd.DataFrame(data= XX1)
XX1_numpy = XX1.to_numpy()

YY1 = {'col1': y1 }
YY1 = pd.DataFrame(data= YY1)
YY1_numpy = YY1.to_numpy()

YY2 = {'col1': y2 }
YY2 = pd.DataFrame(data= YY2)

In [4]:
YY2

,col1
0,540
1,12
2,262
3,451
4,268
5,97
6,400
7,181
8,200
9,271


In [5]:
input = XX1

In [6]:
weights_size = len(input.iloc[0])
solutions_per_problem = 30

pop_size = (solutions_per_problem, weights_size)
new_population = np.random.randint(low = 1, high= 5, size = pop_size)

pop_original = new_population

In [7]:
new_population

array([[1, 2, 3],
       [3, 2, 4],
       [2, 3, 4],
       [4, 2, 1],
       [1, 4, 2],
       [4, 3, 4],
       [1, 1, 4],
       [1, 3, 3],
       [3, 3, 3],
       [4, 2, 4],
       [1, 3, 3],
       [2, 2, 4],
       [2, 1, 4],
       [4, 4, 1],
       [2, 1, 2],
       [1, 2, 3],
       [2, 1, 2],
       [4, 3, 1],
       [4, 2, 2],
       [4, 3, 1],
       [1, 1, 1],
       [4, 4, 1],
       [3, 2, 2],
       [3, 2, 3],
       [2, 1, 2],
       [1, 3, 1],
       [3, 1, 2],
       [1, 2, 2],
       [2, 2, 4],
       [2, 2, 2]])

In [8]:
# fitness of each population:

fitness = np.empty(new_population.shape[0])

def cal_pop_fitness(target_dataframe, electricity_values, pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function calculates the sum of products between each input and its corresponding weight.
    generated_electricity = np.sum(pop*electricity_values, axis=1);
    required_electricity =  target_dataframe['col1']
    error = required_electricity - generated_electricity
    error = error.abs()
    error = error.sum()
    fitness = -error
    
    return fitness


for i in range(len(new_population)):
    fitness[i] = cal_pop_fitness(YY1, XX1, new_population[i])

fitness

array([-3326.,  -534.,     0., -3008., -2358., -2108., -3214., -2246.,
        -712., -1340., -2246., -1080., -2160., -1852., -4544., -3326.,
       -4544., -2336., -1928., -2336., -6790., -1852., -2496., -1350.,
       -4544., -4630., -3540., -4518., -1080., -3464.])

In [9]:
new_population.shape[1]

3

In [10]:
# Select the best parents for mating process:

def select_mating_pool(pop, fitness, number_of_parents):
    parents = np.empty((number_of_parents, new_population.shape[1]))

    for instance in range(number_of_parents):
        max_index = np.where(fitness == np.max(fitness))
        max_index = max_index[0][0]
        parents[instance, :] = pop[instance, :]
        fitness[max_index] = -9999

    return parents

In [11]:
# The mating process with each parent

def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = np.uint8(offspring_size[1]/2)
 
    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring



In [12]:
# Mutations on the offsprings:

def mutation(offspring_crossover):
    
    offspring_operated = offspring_crossover

    # Mutation changes a single gene in each offspring randomly.

    for idx in range(offspring_operated.shape[0]):

        # The random value to be added to the gene.

        random_value = np.random.randint(low = -1, high= 1)

        offspring_operated[idx, 1] = offspring_operated[idx, 1] + random_value

    return offspring_operated


In [13]:
new_population

array([[1, 2, 3],
       [3, 2, 4],
       [2, 3, 4],
       [4, 2, 1],
       [1, 4, 2],
       [4, 3, 4],
       [1, 1, 4],
       [1, 3, 3],
       [3, 3, 3],
       [4, 2, 4],
       [1, 3, 3],
       [2, 2, 4],
       [2, 1, 4],
       [4, 4, 1],
       [2, 1, 2],
       [1, 2, 3],
       [2, 1, 2],
       [4, 3, 1],
       [4, 2, 2],
       [4, 3, 1],
       [1, 1, 1],
       [4, 4, 1],
       [3, 2, 2],
       [3, 2, 3],
       [2, 1, 2],
       [1, 3, 1],
       [3, 1, 2],
       [1, 2, 2],
       [2, 2, 4],
       [2, 2, 2]])

https://towardsdatascience.com/genetic-algorithm-implementation-in-python-5ab67bb124a6 

In [14]:
num_generations = 1000

num_parents_mating = 4

fitness = np.empty(new_population.shape[0])

for generation in range(num_generations):
     # Measuring the fitness of each chromosome in the population.
        
    for i in range(len(new_population)):
        fitness[i] = cal_pop_fitness(YY1, XX1, new_population[i])

    # Selecting the best parents in the population for mating.
    
    parents = select_mating_pool(new_population, fitness, 
                                       num_parents_mating)
 
     # Generating next generation using crossover.
    offspring_crossover = crossover(parents,
                                        offspring_size=(pop_size[0]-parents.shape[0], weights_size))
 
     # Adding some variations to the offsrping using mutation.
    offspring_mutation = mutation(offspring_crossover)
# Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation
    
# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = cal_pop_fitness(equation_inputs, new_population)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = np.where(fitness == numpy.max(fitness))

print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx])

import matplotlib.pyplot
matplotlib.pyplot.plot(best_outputs)
matplotlib.pyplot.xlabel("Iteration")
matplotlib.pyplot.ylabel("Fitness")
matplotlib.pyplot.show()

NameError: name 'ga' is not defined